In [1]:
#default_exp convert

In [1]:
#export
import os, sys
from nbdoc.mdx import get_mdx_exporter
from typing import Union
from nbdev.export import nbglob
from nbconvert.exporters import Exporter
from fastcore.all import Path, parallel, call_parse, bool_arg

# Convert Notebooks To Markdown

> Utilities that help you go from .ipynb -> .md

In [2]:
#export
def nb2md(fname:Union[str, Path], exp:Exporter):
    "Convert a notebook in `fname` to a markdown file."
    file = Path(fname)
    assert file.name.endswith('.ipynb'), f'{str(fname)} is not a notebook.'
    assert file.is_file(), f'file {str(fname)} not found.'
    print(f"converting: {str(file)}")
    try:
        o,r = exp.from_filename(fname)
        file.with_suffix('.md').write_text(o)
        return True
    except Exception as e:
        print(e)
        return False

We can use `nb2md` to convert a notebook to a markdown file with an `Exporter`.  Below, we use the exporter given to us by `nbdoc.mdx.get_mdx_exporter` and use that to create a markdown file from a notebook.

In [4]:
_test_fname = Path('test_files/example_input.ipynb')
_test_dest = Path('test_files/example_input.md')
_test_dest.unlink(missing_ok=True)
assert not _test_dest.exists() # make sure the markdown file doesn't exist

nb2md(fname=_test_fname, exp = get_mdx_exporter()) # create the markdown file
assert _test_dest.exists() # make sure the markdown file does exist
assert len(_test_dest.readlines()) > 10

converting: test_files/example_input.ipynb


In [5]:
!cat {_test_dest}

---
title: my hello page title
description: my hello page description
hide_table_of_contents: true
---


<!--- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! Instead, edit the notebook w/the location & name as this file.-->

## This is a test notebook

This is a shell command:

<CodeSection lang="bash">

<CodeInputBlock>


```bash
echo hello
```

</CodeInputBlock>
<CodeOutputBlock>

    hello


</CodeOutputBlock>

</CodeSection>

We are writing a python script to disk:

<CodeSection lang="py title="myflow.py"">

<CodeInputBlock>


```py title="myflow.py"
from metaflow import FlowSpec, step

class MyFlow(FlowSpec):
    
    @step
    def start(self):
        print('this is the start')
        self.next(self.end)
    
    @step
    def end(self):
        print('this is the end')

if __name__ == '__main__':
    MyFlow()
```

</CodeInputBlock>

</CodeSection>

Another shell command where we run a flow:

<CodeSection lang="bash">

<CodeInputBlock>


```bash
python myflow.py run
```

</C

In [15]:
#export
def parallel_nb2md(basedir:Union[Path,str], exp:Exporter, recursive=True, force_all=False, n_workers=None, pause=0):
    "Convert all notebooks in `dir` to markdown files."
    files = nbglob(basedir, recursive=recursive).filter(lambda x: not x.name.startswith('Untitled'))
    if len(files)==1:
        force_all = True
        if n_workers is None: n_workers=0
    if not force_all:
        # only rebuild modified files
        files,_files = [],files.copy()
        for fname in _files:
            fname_out = fname.with_suffix('.md')
            if not fname_out.exists() or os.path.getmtime(fname) >= os.path.getmtime(fname_out):
                files.append(fname)
    if len(files)==0: print("No notebooks were modified.")
    else:
        if sys.platform == "win32": n_workers = 0
        passed = parallel(nb2md, files, n_workers=n_workers, exp=exp,  pause=pause)
        if not all(passed):
            msg = "Conversion failed on the following:\n"
            print(msg + '\n'.join([f.name for p,f in zip(passed,files) if not p]))

You can use `parallel_nb2md` to recursively convert a directory of notebooks to markdown files.

In [7]:
_test_nbs =  nbglob('test_files/')

In [8]:
#hide
for f in _test_nbs: f.with_suffix('.md').unlink(missing_ok=True)

In [9]:
parallel_nb2md('test_files/', exp=get_mdx_exporter(), recursive=True)

converting: test_files/run_flow_showstep.ipynb
converting: test_files/hello_world.ipynb
converting: test_files/run_flow.ipynb
converting: test_files/example_input.ipynb
converting: test_files/non_executed.ipynbconverting: test_files/writefile.ipynb

converting: test_files/visibility.ipynb
converting: test_files/doc.ipynb
converting: test_files/exec.ipynb
converting: test_files/frontmatter.ipynb


In [10]:
for f in _test_nbs:
    assert f.with_suffix('.md').exists(), f'{str(f)} does not exist.'

The modified times of notebooks are introspected such notebooks that haven't changed after markdown files have been created will not be converted:

In [11]:
parallel_nb2md('test_files/', exp=get_mdx_exporter(), recursive=True)

No notebooks were modified.


However, you can set `force_all` = `True` to force notebooks to convert:

In [12]:
parallel_nb2md('test_files/', exp=get_mdx_exporter(), recursive=True, force_all=True)

converting: test_files/run_flow_showstep.ipynbconverting: test_files/hello_world.ipynb

converting: test_files/run_flow.ipynb
converting: test_files/example_input.ipynb
converting: test_files/writefile.ipynb
converting: test_files/non_executed.ipynb
converting: test_files/visibility.ipynb
converting: test_files/doc.ipynb
converting: test_files/frontmatter.ipynb
converting: test_files/exec.ipynb


In [13]:
#hide
for f in _test_nbs: f.with_suffix('.md').unlink(missing_ok=True)

In [14]:
#export
@call_parse
def nbdoc_build(
    srcdir:str=None,  # A directory of notebooks to convert to docs recursively, can also be a filename.
    force_all:bool_arg=False, # Rebuild even notebooks that havent changed
    n_workers:int=None,  # Number of workers to use
    pause:float=0.5  # Pause time (in secs) between notebooks to avoid race conditions
):
    "Build the documentation by converting notebooks in `srcdir` to markdown"
    parallel_nb2md(basedir=srcdir, 
                   exp=get_mdx_exporter(), 
                   recursive=True, 
                   force_all=force_all, 
                   n_workers=n_workers, 
                   pause=pause)